In [ ]:
from grid.robot.arm.isaac_arm import IsaacArm
from grid.utils.types import Position, Orientation
from grid.model.perception.detection.owlv2 import OWLv2
from grid.utils.types import Velocity
from grid.utils import log

import time
import numpy as np

In [ ]:
agent = IsaacArm()
agent.run()

In [ ]:
obj = "green cube"
    
# Load the object detection model
det_model = OWLv2(use_local=True)

goal_quat_np = np.array([0.0, 0.0, 0.0, 1.0])  # Assuming quaternion interpolation isn't needed
step_size = 0.15  # Define a step size for movement
step_size_z = 0.025  # Define a step size for z movement
focal_length = 24. # Define the focal length of the camera
horizontal_aperture = 20.955 # Define the horizontal aperture of the camera
image_width = 256. # Define the image width
image_height = 256. # Define the image height
t = [1.5, 0, 0.7] # Define the translation vector
q = [0, -0.3, 0, 1] # Define the quaternion
fx = (focal_length / horizontal_aperture) * image_width # Define the focal length in x
fy = (focal_length / horizontal_aperture) * image_height # Define the focal length in y
cx = image_width / 2 # Define the center of the image in x
cy = image_height / 2 # Define the center of the image in y
goal_z = 0.13   # Define the goal z position
# Define the rotation matrix
rot_matrix = np.array([[1 - 2*(q[1]**2 + q[2]**2), 2*(q[0]*q[1] - q[2]*q[3]), 2*(q[0]*q[2] + q[1]*q[3])],
                        [2*(q[0]*q[1] + q[2]*q[3]), 1 - 2*(q[0]**2 + q[2]**2), 2*(q[1]*q[2] - q[0]*q[3])],
                        [2*(q[0]*q[2] - q[1]*q[3]), 2*(q[1]*q[2] + q[0]*q[3]), 1 - 2*(q[0]**2 + q[1]**2)]])
# Define the world to camera transformation matrix
T_world_from_cam = np.array([[rot_matrix[0, 0], rot_matrix[0, 1], rot_matrix[0, 2], t[0]],
                                [rot_matrix[1, 0], rot_matrix[1, 1], rot_matrix[1, 2], t[1]],
                                [rot_matrix[2, 0], rot_matrix[2, 1], rot_matrix[2, 2], t[2]],
                                [0, 0, 0, 1]])

preprocessor_config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/620M [00:00<?, ?B/s]

In [ ]:
def get_box_center(box, image_size):
    """
    Calculate the center of a single bounding box given in xyxy format.

    Args:
        box (list or np.ndarray): A bounding box with format [x_min, y_min, x_max, y_max].
    
    Returns:
        tuple: Center point (x_center, y_center).
    """
    x_min, y_min, x_max, y_max = box
    if image_size[0] * image_size[1] * .2 < (x_max-x_min) * (y_max-y_min):
        return -1, -1
    x_center = (x_min + x_max) / 2
    y_center = (y_min + y_max) / 2
    return x_center, y_center

In [ ]:
# Get the images from the camera
rgb, depth = None, None
while rgb is None or rgb.data is None or depth is None or depth.data is None:
    print ("Waiting for image")
    rgb = agent.getImage("camera_rgb_0")
    depth = agent.getImage("camera_depth_0")
    time.sleep(0.2)
    
log("grid/rgb_img", rgb)
log("grid/depth_img", depth)

data = rgb.data
# Run the object detection model
boxes, scores, labels = det_model.run(rgbimage=data[:, :, :3], prompt=obj)
while boxes is None or len(boxes) == 0:
    boxes, scores, labels = det_model.run(rgbimage=data[:, :, :3], prompt=obj)

i = np.argmax(scores)
# Get the center of the bounding box
center_x, center_y = get_box_center(boxes[i], [data.shape[1], data.shape[0]])
if center_x == -1 and center_y == -1:
    print ("Object too big")

Waiting for image


[2025-05-21T14:39:53Z WARN  re_log_types::path::parse_path] When parsing the entity path "viz/owlv2/detect_green cube": Unescaped whitespace. The path will be interpreted as /viz/owlv2/detect_green\ cube
[2025-05-21T14:39:53Z WARN  re_log_types::path::parse_path] When parsing the entity path "viz/owlv2/detect_green cube/bboxes": Unescaped whitespace. The path will be interpreted as /viz/owlv2/detect_green\ cube/bboxes


In [ ]:
# Convert to 3D coordinates (camera -> world)
depth_data = depth.data[:, :, 0]
Z = depth_data[int(center_y), int(center_x)]
X_c = ((center_x - cx) / fx) * Z
Y_c = ((center_y - cy) / fy) * Z
Z_c = Z

camera_point_h = np.array([X_c, Y_c, Z_c, 1.0])
world_point_h = T_world_from_cam @ camera_point_h
X_w, Y_w, Z_w = world_point_h[:3]

goal_pos_np = np.array([X_w-0.25, Y_w-0.1, Z_w]) # offset for (cube side / 2.)

In [ ]:
# Move the robot to the (x, y) position of the object

# Iterate to reach the goal position
for i in range(250):
    curr_pos = agent.getPosition()
    while curr_pos is None or curr_pos.x_val is None or curr_pos.y_val is None:
        curr_pos = agent.getPosition()
        time.sleep(0.1)

    curr_pos_np = np.array([curr_pos.x_val, curr_pos.y_val, curr_pos.z_val])
            
    # Compute delta position
    direction = (goal_pos_np - curr_pos_np) 
    delta_pos = direction * step_size  # Move step_size or remaining distance
    
    # Move towards the goal with delta position
    agent.moveToDeltaPose(Position(delta_pos[0], delta_pos[1], 0.), Orientation(*goal_quat_np))
    time.sleep(0.1)

In [ ]:
# Move the robot to the goal z position
agent.release()
# Iterate to reach the goal position
for i in range(700):
    curr_pos = agent.getPosition()
    while curr_pos is None or curr_pos.x_val is None or curr_pos.y_val is None:
        curr_pos = agent.getPosition()
        time.sleep(0.1)

    delta = (goal_z - curr_pos.z_val)
    delta_pos = (goal_z - curr_pos.z_val) * step_size_z

    if abs(delta) < 0.01:
        print("Z Reached")
        break
    # Move towards the goal with delta position
    agent.moveToDeltaPose(Position(0., 0., delta_pos), Orientation(*goal_quat_np))
    time.sleep(0.1)

agent.grasp()